# Имплементација експертског система за животиње

Пример из [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

У овом примеру ћемо имплементирати једноставан систем заснован на знању који одређује животињу на основу неких физичких карактеристика. Систем се може представити следећим AND-OR стаблом (ово је део целог стабла, лако можемо додати још неких правила):

![](../../../../../../translated_images/sr/AND-OR-Tree.5592d2c70187f283.webp)


## Наш сопствени експертски систем шкољка са инверзним закључивањем

Хајде да покушамо да дефинишемо једноставан језик за представљање знања заснован на продукционим правилима. Користићемо Python класе као кључне речи за дефинисање правила. У суштини, биће 3 типа класа:
* `Ask` представља питање које треба поставити кориснику. Садржи скуп могућих одговора.
* `If` представља правило, и то је само синтаксички шећер за чување садржаја правила
* `AND`/`OR` су класе које представљају AND/OR гране стабла. Оне само чувају листу аргумената у себи. Да бисмо поједноставили код, сва функционалност је дефинисана у родитељској класи `Content`


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

У нашем систему, радна меморија би садржала листу **чињеница** као **парова атрибут-вредност**. Базу знања можемо дефинисати као један велики речник који мапира акције (нове чињенице које треба убацити у радну меморију) на услове, изражене као AND-OR изрази. Такође, неке чињенице се могу `Питати`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Да бисмо извршили обрнути закључак, дефинисаћемо класу `Knowledgebase`. Она ће садржати:
* Радну `memory` - речник који мапира атрибуте на вредности
* Правила базе знања `rules` у формату као што је горе дефинисано

Две главне методе су:
* `get` за добијање вредности атрибута, извођење инференције ако је потребно. На пример, `get('color')` би добило вредност за поље боје (питаће ако је потребно, и сачуваће ту вредност за каснију употребу у радној меморији). Ако питамо `get('color:blue')`, питаће за боју, а затим вратити вредност `y`/`n` у зависности од боје.
* `eval` извршава саму инференцију, тј. прелази кроз AND/OR стабло, процењује под-циљеве, итд.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Хајде сада да дефинишемо нашу базу знања о животињама и обавимо консултацију. Имајте у виду да ће вам овај позив постављати питања. Можете одговарати тако што ћете уносити `y`/`n` за питања на која се одговара да/не, или тако што ћете назначити број (0..N) за питања са дужим избором одговора.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Коришћење Experta за унапредно извођење

У следећем примеру ћемо покушати да имплементирамо унапредно извођење користећи једну од библиотека за представљање знања, [Experta](https://github.com/nilp0inter/experta). **Experta** је библиотека за креирање система унапредног извођења у Питону, која је дизајнирана да буде слична класичном старом систему [CLIPS](http://www.clipsrules.net/index.html).

Такође бисмо могли да имплементирамо унапредно повезивање сами без већих проблема, али наивне имплементације обично нису веома ефикасне. За ефикасније поклапање правила користи се посебан алгоритам [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Систем ћемо дефинисати као класу која наслеђује `KnowledgeEngine`. Свака правила је дефинисана као посебна функција са анотацијом `@Rule`, која одређује када правило треба да се активира. Унутар правила можемо додати нове чињенице користећи функцију `declare`, а додавање тих чињеница ће резултирати позивом још неких правила од стране механизма за унапредну инференцу.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Једном када дефинишемо базу знања, попуњавамо нашу радну меморију са неким почетним чињеницама, а затим позивамо `run()` методу да изврши закључивање. Као резултат, можете видети да се нове изведене чињенице додају у радну меморију, укључујући коначну чињеницу о животињи (ако смо правилно поставили све почетне чињенице).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Одрицање од одговорности**:  
Овај документ је преведен помоћу АИ услуге за превођење [Co-op Translator](https://github.com/Azure/co-op-translator). Иако дајемо све од себе да буде прецизно, имајте у виду да аутоматизовани преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитетом. За критичне информације препоручује се професионални људски превод. Није нам могуће преузети одговорност за било каква неразумевања или погрешне интерпретације настале употребом овог превода.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
